## Set up

In [1]:
import os
import sys
sys.path.append("../")
from app.core.config import settings


### Granting everyone free credit

In [35]:
import stripe
stripe.api_key = settings.STRIPE_API_KEY

In [40]:
coupon = stripe.Coupon.create(
    name="Free Credit Coupon",
    amount_off=500,
    currency="usd",
    max_redemptions=100,
)


In [51]:
coupon = stripe.Coupon.retrieve(coupon.id)
coupon

<Coupon coupon id=tv8K3lfM at 0x121ad62c0> JSON: {
  "amount_off": 500,
  "created": 1726384846,
  "currency": "usd",
  "duration": "once",
  "duration_in_months": null,
  "id": "tv8K3lfM",
  "livemode": true,
  "max_redemptions": 100,
  "metadata": {},
  "name": "Free Credit Coupon",
  "object": "coupon",
  "percent_off": null,
  "redeem_by": null,
  "times_redeemed": 52,
  "valid": true
}

In [42]:
succedded_customers = []

In [44]:
customers = stripe.Customer.list(limit=100)

for customer in customers:
    invoice = stripe.Invoice.create(
        customer=customer.id,
        auto_advance=False,
        collection_method="charge_automatically",
        discounts=[{"coupon": coupon.id}],
        description="Free credit for your account balance"
    )
    stripe.InvoiceItem.create(
        customer=customer.id,
        price="price_1PyzzgDTmNvCxNQyfT5zr0WC",
        quantity=500,
        invoice=invoice.id,
    )

    stripe.Invoice.finalize_invoice(invoice.id)
    succedded_customers.append(customer.id)

In [47]:
len(customers)

51

In [48]:
len(succedded_customers)

51

In [38]:
coupon.delete()

<Coupon coupon id=2NGGxLyg at 0x107e77b30> JSON: {
  "deleted": true,
  "id": "2NGGxLyg",
  "object": "coupon"
}

### Remove agent usage from the subscription item

In [4]:
customers = stripe.Customer.list(limit=100)

In [5]:
from typing import List

from app.services.stripe.types import Prices


for customer in customers:
    customer = customer.retrieve(customer.id, expand=["subscriptions"])
    subscriptions = customer.subscriptions.data
    for sub in subscriptions:
        sub_items: List[stripe.SubscriptionItem] = sub["items"].data
        sub_item_id = None
        for si in sub_items:
            if si.price.id == "price_1Pbq6QDTmNvCxNQyzLLk30Qb":
                sub_id = sub.id
                sub_item = si
                sub_item_id = si.id
                break
        if sub_item_id:
            break
    if not sub_item_id:
        print(f"Customer {customer.id} does not have the subscription item")
        continue
    try:
        re_sub_item = stripe.SubscriptionItem.retrieve(sub_item_id)
        deleted_sub_item = re_sub_item.delete(clear_usage=True)
    except Exception as e:
        #print(f"Error deleting subscription item for customer {customer.id}: {e}")
        try:
            sub = stripe.Subscription.retrieve(sub_id)
            if len(sub["items"].data) != 1:
                raise ValueError("Subscription has more than one item "+str(sub["items"].data))
            sub.delete()
        except Exception as e:
            print(f"Error retrieving subscription {sub_id}: {e}")

Customer cus_QqmvVm4RiAcLeD does not have the subscription item
Customer cus_Qo9ToX96Ea2KXp does not have the subscription item
Error retrieving subscription sub_1PbqcIDTmNvCxNQyNxns45x8: Request req_PhU1cop6YlTDbH: No such subscription: 'sub_1PbqcIDTmNvCxNQyNxns45x8'


In [25]:
re_sub_item = stripe.SubscriptionItem.retrieve("si_QjzKJnMRI0ylH9")

In [29]:
deleted_sub_item = re_sub_item.delete(clear_usage=True)

### Products and prices for different bots

In [8]:
autores_prod = stripe.Product.create(name="Autoresolution Bot")
checkout_prod = stripe.Product.create(name="Checkout Bot")

In [9]:
print(autores_prod.id)
print(checkout_prod.id)

prod_Ps4NVq304hcUF7
prod_Ps4N4REASAWnHe


In [11]:
autores_voice = stripe.Price.create(nickname="Autoresolution per minute of voice chat",
                                    product="prod_Ps4NVq304hcUF7",
                                    currency="usd",
                                    unit_amount=15,
                                    recurring={"interval":"month", "usage_type":"metered"},
                                    billing_scheme="per_unit",
                                    )
autores_convo = stripe.Price.create(nickname="Autoresolution per chat conversation",
                                    product="prod_Ps4NVq304hcUF7",
                                    currency="usd",
                                    unit_amount=5,
                                    recurring={"interval":"month", "usage_type":"metered"},
                                    billing_scheme="per_unit",
                                    )

checkout_voice= stripe.Price.create(nickname="Checkout per minute of voice chat",
                                    product="prod_Ps4N4REASAWnHe",
                                    currency="usd",
                                    unit_amount=10,
                                    recurring={"interval":"month", "usage_type":"metered"},
                                    billing_scheme="per_unit",
                                    )

checkout_convo = stripe.Price.create(nickname="Checkout per chat conversation",
                                    product="prod_Ps4N4REASAWnHe",
                                    currency="usd",
                                    unit_amount=3,
                                    recurring={"interval":"month", "usage_type":"metered"},
                                    billing_scheme="per_unit",
                                    )


In [7]:
print(autores_voice.nickname,  autores_voice.unit_amount, autores_voice.id)
print(autores_convo.nickname, autores_convo.unit_amount,  autores_convo.id)
print(checkout_voice.nickname, checkout_voice.unit_amount, checkout_voice.id)
print(checkout_convo.nickname, checkout_convo.unit_amount, checkout_convo.id)

Autoresolution per minute of voice chat 15 price_1OwyTMDTmNvCxNQynpS7a0Oo
Autoresolution per chat conversation 5 price_1OwyTMDTmNvCxNQyQDVUzTvw
Checkout per minute of voice chat 10 price_1OwyTMDTmNvCxNQynVp8R6Fa
Checkout per chat conversation 3 price_1OwyTMDTmNvCxNQypUdrQiPZ


### Assistant product and usage

In [3]:
agent_prod = stripe.Product.create(name="Agent")

In [4]:
agent_prod_price = stripe.Price.create(nickname="Agent usage",
                                    product=agent_prod.id,
                                    currency="usd",
                                    unit_amount=1,
                                    recurring={"interval":"month", "usage_type":"metered"},
                                    billing_scheme="per_unit",
                                    )

## Create customer and their subscription

In [12]:
vitranna_customer = stripe.Customer.create(name="Vitrinnea")

In [13]:
vitranna_customer.id

'cus_Ps4OVilG7ZN8DN'

In [22]:
vitranna_customer = stripe.Customer.retrieve("cus_Ps4OVilG7ZN8DN", expand=["subscriptions"])

In [24]:
from typing import List
items = vitranna_customer.subscriptions.data[0]["items"]

for si in items.data:
    print(si.id)



si_Ps4R4OcvQgiEO8
si_Ps4RJfECTGf9Lf
si_Ps4RWjxauLzPZ1
si_Ps4R6CGIItt20U


In [23]:
vitranna_customer.subscriptions.data

[<Subscription subscription id=sub_1P2KJ2DTmNvCxNQy39fQrhx5 at 0x121be1ae0> JSON: {
   "application": null,
   "application_fee_percent": null,
   "automatic_tax": {
     "enabled": false,
     "liability": null
   },
   "billing_cycle_anchor": 1712352496,
   "billing_cycle_anchor_config": null,
   "billing_thresholds": null,
   "cancel_at": null,
   "cancel_at_period_end": false,
   "canceled_at": null,
   "cancellation_details": {
     "comment": null,
     "feedback": null,
     "reason": null
   },
   "collection_method": "charge_automatically",
   "created": 1712352496,
   "currency": "usd",
   "current_period_end": 1714944496,
   "current_period_start": 1712352496,
   "customer": "cus_Ps4OVilG7ZN8DN",
   "days_until_due": null,
   "default_payment_method": null,
   "default_source": null,
   "default_tax_rates": [],
   "description": null,
   "discount": null,
   "discounts": [],
   "ended_at": null,
   "id": "sub_1P2KJ2DTmNvCxNQy39fQrhx5",
   "invoice_settings": {
     "account_

In [11]:
vitranna_customer.id

'cus_PmSfJ4zzkwIoRl'

In [16]:
sub = stripe.Subscription.create(
  customer=vitranna_customer.id,
  items=[{"price":autores_convo.id},
         {"price":autores_voice.id},
         {"price":checkout_convo.id},
         {"price":checkout_voice.id},],
)

In [21]:
sub.id

'sub_1P2KJ2DTmNvCxNQy39fQrhx5'

## Add phone number to stripe

In [3]:
pn_prod = stripe.Product.create(name="Phone Number")

In [5]:
pn_price = stripe.Price.create(nickname="Phone Number",
                                    product=pn_prod.id,
                                    currency="usd",
                                    unit_amount=200,
                                    recurring={"interval":"month", "usage_type":"licensed"},
                                    billing_scheme="per_unit",
                                    )

In [ ]:
pn_price

In [8]:
vitranna_customer = stripe.Customer.retrieve("cus_PmSfJ4zzkwIoRl", expand=["subscriptions"])

#### Because this is a liscened usage, there needs to be payment method.

In [ ]:
stripe.Customer.modify(vitranna_customer.id, email="max@syntag.org")

In [18]:
pn_sub = stripe.Subscription.create(
    customer=vitranna_customer.id,
    items=[{"price":pn_price.id, "quantity":1}],
    collection_method="send_invoice",
    days_until_due=7,
    payment_settings={"payment_method_types":["customer_balance"]},
)

In [19]:
pn_sub.id

'sub_1P4CxvDTmNvCxNQykWpV7cEn'

## Add subscription to all stripe customers

In [23]:
customer_list = stripe.Customer.list(limit=100, expand=["data.subscriptions", "data.invoice_settings"])
while True:
    for c in customer_list:
        for s in c.subscriptions.get("data"):
            items = s.get("items").get("data")
            for si in items:
                if si.price.id == settings.STRIPE_AGENT_USAGE_PRICE_ID:
                    continue
                    # stripe.Subscription.cancel(s.get("id"))
                    # print("Cancelled agent usage for", c.email)
        if not c.default_source and not c.invoice_settings.default_payment_method:
            pm_link = stripe.PaymentMethod.attach("pm_card_visa", customer=c.id)
            stripe.Customer.modify(c.id, invoice_settings={"default_payment_method":pm_link.id})
            print("Added card to", c.email)

        sub = stripe.Subscription.create(
            customer=c.id,
            items=[{"price": settings.STRIPE_AGENT_USAGE_PRICE_ID}],
        )
        print("Added agent usage to", c.email)
    if not customer_list.has_more:
        break
    customer_list = stripe.Customer.list(limit=100, starting_after=customer_list.data[-1].id)

Added agent usage to maxtmp4+clerk_test@gmail.com
Added agent usage to maxtmp3+clerk_test@gmail.com
Added agent usage to maxtmp2+clerk_test@gmail.com
Added agent usage to maxtmp+clerk_test@gmail.com
Added agent usage to max1+clerk_test@gmail.com
Added agent usage to vivek@syntag.org
Added agent usage to vikram.a.khandelwal@gmail.com
Added agent usage to nadig.vivek@gmail.com
Added agent usage to nadig.vivek@gmail.com
Added agent usage to nadig.vivek@gmail.com
Added agent usage to nadig.vivek@gmail.com
Added agent usage to vivek@syntag.ai
Added agent usage to vikram@syntag.ai
Added agent usage to kaufmann@terpmail.umd.edu
Added agent usage to max906993@gmail.com
Added agent usage to max2+clerk_test@gmail.com
Added agent usage to max@syntag.org
